In [34]:
#We are going to fetch the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [35]:
#We can see there are 70.000 images, with 784 features each one
X, y = mnist["data"], mnist["target"]
X.shape


(70000, 784)

In [36]:
y.shape

(70000,)

In [40]:
#The MNIST dataset has already the test set:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### Binary detector

In [41]:
#We are going to identify the digit 5, so we create first the target vectors for the classification task
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)